In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv('data.csv', sep=',')
data.pop('Lp.')
data.head()

In [ ]:
def gain_type_concordance(a, b, p, q):
    # Współczynnik zgodności dla kryterium typu zysk
    if a - b >= -q:
        return 1
    elif a - b < -p:
        return 0
    else:
        return (p - (b - a))  / (p - q)
    
def cost_type_concordance(a, b, p, q):
    # Współczynnik zgodności dla kryterium typu koszt
    if a - b <= q:
        return 1
    elif a - b > p:
        return 0
    else:
        return (p - (a - b)) / (p - q)

def count_marginal_concordace(p, q):
    struct = np.zeros((data.shape[0], data.shape[0], data.shape[1]))

    for i, a in enumerate(data.values):
        for j, b in enumerate(data.values):
            if i == j:
                struct[i, j, :] = 1
                continue
            for k, g in enumerate(a):
                if k != 1:
                    struct[i, j, k] = gain_type_concordance(g, b[k], p[k], q[k])
                else:
                    struct[i, j, k] = cost_type_concordance(g, b[k], p[k], q[k])

    return struct

c = count_marginal_concordace([200, 2, 1000, 1000], [100, 4, 500, 500])

In [ ]:
def gain_type_discordance(a, b, p, v):
    # Współczynnik niezgodności dla kryterium typu zysk
    if a - b <= -v:
        return 1
    elif a - b >= -p:
        return 0
    else:
        return ((b - a) - p)  / (v - p)
    

def cost_type_discordance(a, b, p, v):
    # Współczynnik niezgodności dla kryterium typu koszt
    if a - b >= v:
        return 1
    elif a - b <= p:
        return 0
    else:
        return (v - (a - b)) / (v - p)
    

def count_marginal_discordace(p, v, c):
    struct = np.zeros((data.shape[0], data.shape[0], data.shape[1]))

    for i, a in enumerate(data.values):
        for j, b in enumerate(data.values):
            if i == j:
                struct[i, j, :] = 0
                continue
            for k, g in enumerate(a):
                if k != 1:
                    struct[i, j, k] = gain_type_discordance(g, b[k], p[k], v[k]) if c[i, j, k] == 0 else 0
                else:
                    struct[i, j, k] = cost_type_discordance(g, b[k], p[k], v[k]) if c[i, j, k] == 0 else 0

    return struct
    
d = count_marginal_discordace([200, 2, 1000, 1000], [300, 6, 1500, 1500], c)

In [ ]:
weights = [4, 4, 2, 3]

def count_total_concordance(c, w):
    return np.sum(c * w, axis=2) / np.sum(w)


C = count_total_concordance(c, weights)

In [ ]:
def count_outranking_credibility(C, d):
    struct = np.zeros_like(C)
    for i in range(data.shape[0]):
        for j in range(data.shape[0]):
            f = np.where(d[i, j, :] > C[i, j])
            if len(f[0]) == 0:
                struct[i, j] = C[i, j]
            else:
                if np.any(d[i, j, f] == 1):
                    struct[i, j] = 0
                else:
                    struct[i, j] = C[i, j] * np.prod((1 - d[i, j, f]) / (1 - C[i, j]))
    return struct

R = count_outranking_credibility(C, d)